Conitnuación del notebook pre_processing_data_jesus. Se añaden features adicionales para seguir haciendo pruebas

In [135]:
import pandas as pd
from fuzzywuzzy import fuzz


In [136]:
tweets = pd.read_csv("data/train_pre_processing.csv")
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   keyword               7552 non-null   object 
 1   location              5080 non-null   object 
 2   text                  7613 non-null   object 
 3   keyword_grouped       7552 non-null   object 
 4   text_contain_keyword  7552 non-null   object 
 5   total_words           7613 non-null   int64  
 6   len_text              7613 non-null   int64  
 7   total_upper_chars     7613 non-null   int64  
 8   total_numbers_chars   7613 non-null   int64  
 9   total_special_chars   7613 non-null   int64  
 10  total_common_chars    7613 non-null   int64  
 11  contain_question      7613 non-null   bool   
 12  contain_link          7613 non-null   bool   
 13  contain_hashtag       7613 non-null   bool   
 14  contain_upper_words   7613 non-null   bool   
 15  total_3_words        

Convertir text_contain_keyword en bool (en el anterior no se hizo). Aquellos registros con null se sustituyen por false

In [137]:
tweets['text_contain_keyword'] = tweets.text_contain_keyword.fillna(False).astype('bool')

El texto cointiene al menos una de las palabras del campo location (la palabra debe al menos tener 4 caracteres)

In [138]:
def string_contain_word_string(s,ws):
    if pd.isna(ws):
        return False
    for w1 in ws.lower().split(" "):
        for w2 in w1.split(","):
            for w3 in w2.split(":"):
                for w in w3.split("."):
                    if len(w) > 3 and w in s.lower():
                        return True
    return False

tweets['text_contain_word_location'] = tweets.apply(lambda x: string_contain_word_string(x.text,x.location), axis = 1)


In [139]:
tweets.text_contain_word_location.value_counts()

False    7352
True      261
Name: text_contain_word_location, dtype: int64

Longitud del campo location siendo 0 el valor por default para aquellos que no poseen dicho campo

In [140]:
tweets['len_location_cero_default'] = tweets.location.transform(lambda x: 0 if pd.isna(x) else len(x))

Longitud del campo location siendo el promedio el valor por default para aquellos que no poseen dicho campo

In [141]:
len_location_mean = int(tweets.loc[tweets.len_location_cero_default != 0,'len_location_cero_default'].mean())

tweets['len_location_mean_default'] = tweets.len_location_cero_default.replace(0, len_location_mean)


Total de palabras en location  siendo 0 el valor por default para aquellos que no poseen dicho campo

In [142]:
tweets['total_words_location_cero_default'] = tweets.location.transform(lambda x: 0 if pd.isna(x) else len(x.split(" ")))


Total de palabras en location  siendo el promedio el valor por default para aquellos que no poseen dicho campo

In [143]:
total_words_location_mean = int(tweets.loc[tweets.total_words_location_cero_default != 0,'total_words_location_cero_default'].mean())

tweets['total_words_location_mean_default'] = tweets.total_words_location_cero_default.replace(0, total_words_location_mean)


Similarmente a text_contain_keyword, el siguiente campo indica si el texto contiene una palabra muy similar según la librería fuzzywuzzy 

In [144]:
def text_contain_similarity_word(x,y):
    if pd.isna(y):
        return False
    
    for w in x.lower().split(" "):
        if fuzz.ratio(w,y.lower()) > 80:
            return True
    return False 

tweets['text_contain_keyword_similarity'] = tweets.apply(lambda x: text_contain_similarity_word(x.text,x.keyword_grouped), axis = 1)


Similaridad entre keyword y text (0 por default)

In [145]:
tweets['text_similarity_keyword'] = tweets.apply(lambda x: 0 if pd.isna(x.keyword_grouped) else fuzz.ratio(x.text.lower(),x.keyword_grouped.lower()), axis = 1)


mejor similaridad entre keyword y cada palabra del text (0 por default)

In [146]:
def get_best_similarity(s,w):
    similarity = 0
    
    for x in s.split(" "):
        if fuzz.ratio(x,w) > similarity:
            similarity = fuzz.ratio(x,w)
    return similarity

tweets['text_best_similarity_keyword'] = tweets.apply(lambda x: 0 if pd.isna(x.keyword_grouped) else get_best_similarity(x.text.lower(),x.keyword_grouped.lower()), axis = 1)


Similaridad entre location y text (0 por default)

In [147]:
tweets['text_similarity_location'] = tweets.apply(lambda x: 0 if pd.isna(x.location) else fuzz.ratio(x.text.lower(),x.location.lower()), axis = 1)


mejor similaridad entre location y cada palabra del text (0 por default)

In [148]:
tweets['text_best_similarity_location'] = tweets.apply(lambda x: 0 if pd.isna(x.location) else get_best_similarity(x.text.lower(),x.location.lower()), axis = 1)


Proporción entre cantidad de palabras cortas (menor o igual a tres caracteres) y palabras largas (al menos 5 caracteres)

In [149]:
tweets['ratio_short_big_words']  = tweets['total_3_orless_words'] / tweets['total_5_ormore_words'].replace(0,1)


In [150]:
target = tweets.target
tweets.drop(columns=['target'], inplace=True)
tweets['target'] = target
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   keyword                            7552 non-null   object 
 1   location                           5080 non-null   object 
 2   text                               7613 non-null   object 
 3   keyword_grouped                    7552 non-null   object 
 4   text_contain_keyword               7613 non-null   bool   
 5   total_words                        7613 non-null   int64  
 6   len_text                           7613 non-null   int64  
 7   total_upper_chars                  7613 non-null   int64  
 8   total_numbers_chars                7613 non-null   int64  
 9   total_special_chars                7613 non-null   int64  
 10  total_common_chars                 7613 non-null   int64  
 11  contain_question                   7613 non-null   bool 

In [152]:
#tweets.to_csv('data/train_pre_processing_2.csv', index=False)